In [ ]:
# Understanding the business - This as the name suggests, in-depth analysis of the business objectives and needs has to be done. Current situation must be accessed and from these insights, the goals of carrying out the processes must be defined.

#Understanding the data - A thorough understanding regarding the data should be established. Different attributes of data are examined and facts should be derived from them. At each stage of examining the data completeness and consistency should be verified.

#Preparation of data - This process involves setting up the data in a way such that all the business questions posed can be answered. Exploration of information may be executed for noticing the patterns in light of business understandings.

#Modelling - Selection of modelling technique is the very first step to take followed by the generation of test scenario for validating the model’s quality. After that few more models are generated. All the models are then assessed to make sure that they fall in line with the business initiatives.

#Evaluation - Here the results of models are evaluated in the backdrop of business intentions. Then new objectives may sprout up owing to the new patterns discovered. This is, in fact, an iterative process and the decision whether to consider them or not has to be made in this step before moving further.


## Setup notebook

### Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_predict
import plotly.express as px
import seaborn as sns
%matplotlib inline

## Read Boston Airbnb data
df = pd.read_csv('./data/boston_airbnb/listings.csv')
df.head()

In [ ]:
#Display the number of rows and columns in the data read, and show the names of all the data columns
print(df.shape)
print(df.columns)

In [ ]:
#Plot the fraction of null values for each data column to determine cut off point for columns to drop from the data
vals = df.isnull().mean().sort_values(ascending=False)
(vals[:30]).plot(kind="bar", title="Fraction null values per column", xlabel="Data Column", ylabel="Fraction null values");

In [ ]:
#Display the columns with more than 50% missing values
missing_cols = set(df.columns[df.isnull().mean()>0.5])
print(missing_cols)

In [ ]:
#Remove the columns with more than 50% missing values, and display the number of rows and columns remaining in the data
df1 = df.drop(missing_cols, axis=1, inplace=False)
print(df1.shape)

In [ ]:
#Replace the "$"-sign in price column, and convert the column to numeric type
df2 = df1.copy()

print(df2['price'].head())

df2['price'] =  df2['price'].replace({'\$':''}, regex = True)
df2['price'] =  pd.to_numeric(df2['price'] ,errors='coerce')

print(df2['price'].head())

In [ ]:
#Determine the fraction of values in the prices column this are null
print(df2.shape)
print(df2['price'].isnull().mean())

In [ ]:
#Drop the rows from the data with null values for price, and display the number of rows and columns remaining
df3 = df2.dropna(subset=['price'], axis=0)
print(df3.shape)
print(df3['price'].isnull().mean())

In [ ]:
#Display descriptive statistics of price
df3['price'].describe()

In [ ]:
#Display a histogram of prices per listing
df3['price'].hist();

In [ ]:
#Drop the rows with prices higher than $400, and display the number of rows and columns remaining
df4 = df3[df3['price']<=400]
print(df4.shape)

In [ ]:
#Display a histogram of prices per listing of the data after removing listings with prices more than $400
df4['price'].hist();

In [ ]:
# Question 1: ?

## Reset the data frame by reading and cleaning the data

df = pd.read_csv('./data/boston_airbnb/listings.csv')

#Remove columns with more than 50% missing values
missing_cols = set(df.columns[df.isnull().mean()>0.5])
df = df.drop(missing_cols, axis=1, inplace=False)

#Replace "$" sign in price column, convert to numeric
df['price'] = df['price'].replace({'\$':''}, regex = True)
df['price'] = pd.to_numeric(df['price'] ,errors='coerce')

#Drop rows with null prices
df.dropna(subset=['price'], axis=0)

#Drop rows with prices more than $400
df = df[df['price']<=400]

#Display number of rows and columns, and the column names of the remaining data
print(df.shape)
print(df.columns)

In [ ]:
#Get the total number of listings per host and plot on a bar chart for the top 20 hosts
vals = df['host_name'].value_counts()
((vals[:20])).plot(kind="bar", title="Number of listings per host", xlabel="Host name", ylabel="Number of listings");

In [ ]:
#Display the total number of listings per host
print(vals)

In [ ]:
#Display a histogram of the total number of listings per host, setting the bins from 1 to 10 in steps of 1
vals.hist(bins=[1,2,3,4,5,6,7,8,9,10]);

In [ ]:
#Filter the data to in include hosts with more than one listing only
df_1 = df.groupby('host_name').filter(lambda x: len(x) > 1) 

#Get the number of listings, mean listing price, and mean review score per host
df_1 = df_1.groupby(['host_name']).agg({'id':"count", 'price':"mean", 'review_scores_rating':"mean"})

#Rename the id column to count that contains the number of listings per host
df_1 = df_1.rename(columns={'id': 'count'},inplace=False)

#Sort the data in descending order by the number of listings per host
df_1 = df_1.sort_values("count", ascending=False)

#Display the data
print(df_1)

In [ ]:
#Display a histogram of mean review score per host
df_1['review_scores_rating'].hist();

In [ ]:
#Plot the mean review score per host in ascending order for the 20 hosts with the lowest scores
df_1 = df_1.sort_values("review_scores_rating", ascending=True)
((df_1['review_scores_rating'][:20])).plot(kind="bar", title="Mean review score of listings per host", xlabel="Host name", ylabel="Review score");

In [ ]:
#Display the data sorted by review scores in ascending order
print(df_1.head(10))

In [ ]:
#Display a histogram of mean price per host
df_1['price'].hist();

In [ ]:
#Plot the mean price per host in descending order of the 20 hosts with the highest mean prices
df_1 = df_1.sort_values("price", ascending=False)
((df_1['price'][:20])).plot(kind="bar", title="Mean price of listings per host", xlabel="Host name", ylabel="Price");

In [ ]:
#Sort and displaythe data mean price per host in descending order
df_1 = df_1.sort_values("price", ascending=False)
print(df_1.head(20))

In [ ]:
#Plot the mean price and review scores per host against each other
sns.set_theme(style="ticks")
sns.pairplot(df_1[['price','review_scores_rating']]);


In [ ]:
#Display the descriptive statistics of the number of listings, mean price and mean review score per host
df_1.describe()

In [ ]:
# Question 2: ?

## Reset the data frame by reading and cleaning the data
df = pd.read_csv('./data/boston_airbnb/listings.csv')

#Remove columns with more than 50% missing values
missing_cols = set(df.columns[df.isnull().mean()>0.5])
df = df.drop(missing_cols, axis=1, inplace=False)

#Replace "$" sign in price column, convert to numeric
df['price'] = df['price'].replace({'\$':''}, regex = True)
df['price'] = pd.to_numeric(df['price'] ,errors='coerce')

#Drop rows with null prices
df.dropna(subset=['price'], axis=0)

#Drop rows with prices more than $400
df = df[df['price']<=400]

#Display number of rows and columns, and the column names of the remaining data
print(df.shape)
print(df.columns)

In [ ]:
#Get the number of listings, mean listing price, and mean review score per neighbourhood
df_1 = df.groupby(['neighbourhood']).agg({'id':"count", 'price':"mean", 'review_scores_rating':"mean", 'number_of_reviews':"mean", 'latitude':"mean",'longitude':"mean"})

#rename the id column to count that contains the number of listings per neighbourhood
df_1 = df_1.rename(columns={'id': 'count'},inplace=False)

#sort the data in descending order by the number of listings per neighbourhood
df_1 = df_1.sort_values('count', ascending=False)

#Display the data, and the number of rows and columns of the data
print(df_1)
print(df_1.shape)

In [ ]:
#Get the total number of listings per neighbourhood and plot on a bar chart for the top 20 neighbourhoods
df_1 = df_1.sort_values('count', ascending=False)
((df_1['count'])).plot(kind="bar", title="Number of listings per neighbourhood", xlabel="Neighbourhood", ylabel="Number of listings");

In [ ]:
#Display a histogram of number of listing per neighbourhood
df_1['count'].hist();

In [ ]:
#Plot the mean listing price  per neighbourhood in descending order
df_1 = df_1.sort_values("price", ascending=False)
((df_1['price'])).plot(kind="bar", title="Mean price of listings per neighbourhood", xlabel="Neighbourhood", ylabel="Price");

In [ ]:
#Display the data sorted by the mean listing price per neighbourhood in descending order
print(df_1)

In [ ]:
#Display a histogram of the mean listing price per neighbourhood
df_1['price'].hist();

In [ ]:
#Plot the mean review score per neighbourhood in descending order
df_1 = df_1.sort_values("review_scores_rating", ascending=False)
((df_1['review_scores_rating'])).plot(kind="bar", title="Mean review score of listings per neighbourhood", xlabel="Neighbourhood", ylabel="Review score");

In [ ]:
#Display the data sorted by the mean review score per neighbourhood in descending order
print(df_1)

In [ ]:
#Display a histogram of the mean review score per neighbourhood
df_1['review_scores_rating'].hist();

In [ ]:
#Plot the total number of listings, mean price, and mean review score for listings per neighbourhood against each other
sns.set_theme(style="ticks")
sns.pairplot(df_1[['count','price','review_scores_rating']]);

In [ ]:
#Visualize the data by location by plotting the neighbourhood data using the mean latitude and longitude of the listings in each neighbourhood.
#Colour scale indicates the mean price, and the size indicates the mean review score of listings per neighbourhood.

df_1 = df_1.sort_values("review_scores_rating", ascending=False)
sns.scatterplot(data=df_1, y="latitude", x="longitude", hue="review_scores_rating",size="count", sizes=(50, 500), palette="deep", legend=False);

In [ ]:
#Visualize the data by location by plotting the neighbourhood data using the mean latitude and longitude of the listings in each neighbourhood.
#Colour scale indicates the mean price, and the size indicates the total number of listings per neighbourhood.

# Reset dataframe index to the neighbourhood index column to a data column
df_2 = df_1.reset_index()

#Key for the map
mapbox_key = 'pk.eyJ1IjoiamFjcXVlc211bGxlciIsImEiOiJja25wczZrMGswY2hkMnFuenIxdWtoNXRtIn0.yV19gFMMxl4AJSVsk7My5Q'

#Plot on a Plotly Express plot with mapbox basemap
fig = px.scatter_mapbox(data_frame=df_2, lat="latitude", lon="longitude", color="price", size="count", hover_name='neighbourhood', zoom=10, size_max=30)

#Style the plot
fig.update_layout(mapbox_style="light", mapbox_accesstoken=mapbox_key);

fig.show();

In [ ]:
df_1.describe()

In [ ]:
# Question 3: 

## Reset the data frame by reading and cleaning the data
df = pd.read_csv('./data/boston_airbnb/listings.csv')

#Remove columns with more than 50% missing values
missing_cols = set(df.columns[df.isnull().mean()>0.5])
df = df.drop(missing_cols, axis=1, inplace=False)

#Replace "$" sign in price column, convert to numeric
df['price'] = df['price'].replace({'\$':''}, regex = True)
df['price'] = pd.to_numeric(df['price'] ,errors='coerce')

#Drop rows with null prices
df.dropna(subset=['price'], axis=0)

#Drop rows with prices more than $400
df = df[df['price']<=400]

#Display number of rows and columns, and the column names of the remaining data
print(df.shape)
print(df.columns)

In [ ]:
#Select only specific data columns for which the regression will be performed (numeric and categorical columns)
df_1 = df[['accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 'guests_included', 'number_of_reviews', 'bed_type' ]]

#Display the selected data, and its number of rows and columns
print(df_1.head())
print(df_1.shape)

In [ ]:
#Get and display a list of the categorical data columns - only bed_type
cat_df = df_1.select_dtypes(include=['object'])
cat_cols_lst = cat_df.columns
print(cat_cols_lst)

In [ ]:
#Count and display the number of listings for each bed_type category value
vals = df_1['bed_type'].value_counts()
print(vals)

In [ ]:
#Function to remove the category columns, and create a column for each category with a numerical value to indicate the category
def create_dummy_df(df, cat_cols, dummy_na):
    '''
    INPUT:
    df - the pandas dataframe in which the category columns should be added
    cat_cols - list of the names of the category columns in df
    dummy_na - boolean to indicate if the rows with null values should be removed
    
    OUTPUT:
    df - a dataframe of with the specified category columns removed, and a column created for each category with a numerical value to indicate the category
    '''
    for col in  cat_cols:
        try:
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], prefix=col, prefix_sep='_', drop_first=True, dummy_na=dummy_na)], axis=1)
        except:
            continue
    return df

In [ ]:
#Change the bed_type categorical column into a numerical column with a 0 or 1 to indicate the bed type per listing
df_2 = create_dummy_df(df_1, cat_cols_lst, False)
print(df_2)

In [ ]:
#Determine and plot the fraction of null values for each column in the data
vals = df_2.isnull().mean().sort_values(ascending=False)
(vals).plot(kind="bar");

In [ ]:
#Remove the rows with a null value in any of the columns, and display the number of rows and columns of the data now
df_3 = df_2.dropna()
df_3.shape


In [ ]:
#Display the correlation coefficients between the variables in the data on a heat map
sns.heatmap(df_3.corr(), annot=True, fmt=".2f");

In [ ]:
#Display histograms of all the variables in the data
df_3.hist();


In [ ]:
#Display correlation plots of price between all the other variables (columns)
sns.set_theme(style="ticks")
sns.pairplot(data=df_3, y_vars='price', x_vars=['accommodates', 'bathrooms', 'bedrooms', 'beds',
       'guests_included', 'number_of_reviews', 'bed_type_Couch',
       'bed_type_Futon', 'bed_type_Pull-out Sofa', 'bed_type_Real Bed'] )

In [ ]:
#Function to perform linear regression between response variable and a list of explanatory variables
def fit_linear_mod(df, response_col, explanatory_cols, test_size=.3, rand_state=42):
        
    '''
    INPUT:
    df - the pandas dataframe that contains all the data
    response_col - the column name of the response variable in df
    explanatory_cols - list of the column names of the explanatory variables in df
    
    OUTPUT:
    test_score - regression coefficient obtained for the test data
    train_score - regression coefficient obtained for the training data
    lm_model - linear regression model instance
    X_train - explanatory data values for training
    X_test - explanatory data values for testing
    y_train - response data values for training
    y_test - response data values for testing
    '''

    y = df[response_col]
    X = df[explanatory_cols]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)
   
    lm_model = LinearRegression(normalize=True) 
    lm_model.fit(X_train, y_train)

    y_test_preds = lm_model.predict(X_test)
    y_train_preds = lm_model.predict(X_train)

    test_score = r2_score(y_test, y_test_preds)
    train_score = r2_score(y_train, y_train_preds)

    return test_score, train_score, lm_model, X_train, X_test, y_train, y_test

In [ ]:
#Perform regression between price and all the other explanatory variables in the data, and display the regression coefficients for the traiing and test data
response_col = 'price'
explanatory_cols = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'guests_included', 'number_of_reviews',
                    'bed_type_Couch', 'bed_type_Futon', 'bed_type_Pull-out Sofa', 'bed_type_Real Bed']

test_score, train_score, lm_model, X_train, X_test, y_train, y_test = fit_linear_mod(df_3, response_col, explanatory_cols)

print(train_score)
print(test_score)

In [ ]:
#Perform regression between price and only the  explanatory variables with high correlation coefficients with price, and display the regression coefficients for the traiing and test data
response_col = 'price'
explanatory_cols = ['accommodates', 'bedrooms', 'beds' ]

test_score, train_score, lm_model, X_train, X_test, y_train, y_test = fit_linear_mod(df_3, response_col, explanatory_cols)

print(train_score)
print(test_score)

In [ ]:
#Function to put the linear regression equation's coefficients into a data frame
def coef_weights(coefficients, X_train):
    '''
    INPUT:
    coefficients - linear model coefficients
    X_train - exlanatory data for training
    
    
    OUTPUT:
    coefs_df - a dataframe with the explanatory variable names, coefficients, and absolute values of the coefficients (for sorting)
    '''

    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = lm_model.coef_
    coefs_df['abs_coefs'] = np.abs(lm_model.coef_)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df

In [ ]:
#Determine and display the coefficients of the linear equation
coef_df = coef_weights(lm_model.coef_, X_train)
print(coef_df)

In [ ]:
#Create and display the predicted prices, the actual prices, and the differences for the training and test data

y_test_preds = lm_model.predict(X_test)
preds_vs_act = pd.DataFrame(np.hstack([y_test.values.reshape(y_test.size,1), y_test_preds.reshape(y_test.size,1)]))
preds_vs_act.columns = ['actual', 'preds']
preds_vs_act['diff'] = preds_vs_act['actual'] - preds_vs_act['preds']

print(preds_vs_act)

In [ ]:
#Plot the actual prices against the predicted values using the linear regression equation
plt.plot(preds_vs_act['preds'], preds_vs_act['actual'], 'bo');
plt.xlabel('predicted');
plt.ylabel('actual');
plt.title('Actual price vs. Predicted price')